In [ ]:
#!pip install torch
import sqlite3
import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np

In [ ]:
import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split

In [ ]:
# Создаем соединение с базой данных
conn = sqlite3.connect('garpix_db_0.0.2.db')

In [ ]:
# Формируем DataFrame  
df = pd.read_sql_query("""
    SELECT
        dr.calculation_id,
		b.size_width, 
		b.size_height, 
		b.size_length,
""", conn)
print(df.head())

In [ ]:
grouped_df = df.groupby('calculation_id').agg(list)
grouped_df.groupby('calculation_id').agg(pd.Series.tolist)
grouped_df = df.groupby('calculation_id')[['size_width', 'size_height', 'size_length']].agg(list)

In [ ]:
temp_index = [index for index in grouped_df['size_width'].index]
temp_index

In [ ]:
def get_data_index_1(index):
	temp_1 = []
	len_temp = len(grouped_df['size_width'][index])
	for len_x in range(len_temp):
		temp_1 = temp_1 + [grouped_df['size_width'][index][len_x], grouped_df['size_height'][index][len_x], grouped_df['size_length'][index][len_x]]
	return temp_1

In [ ]:
# Наконец, создаем список тензоров из этих списков значений size_width.
data_x =  [torch.tensor(get_data_index_1(index)) for index in temp_index] # + \
    #      [torch.tensor(get_data_index_2(index)) for index in temp_index] + \
     #     [torch.tensor(get_data_index_3(index)) for index in temp_index]
for x in range(0, len(data_x)):
	data_x[x] = data_x[x].to(torch.float)
	data_x[x] = data_x[x] / data_x[x].max()


In [ ]:
data_y = pad_sequence(data_x, batch_first=True)

In [ ]:
resuld_df = df.groupby('calculation_id')['density_percent'].agg(list)

In [ ]:
# Создаем np.array уникальных значений filling_space_percent для каждого calculation_id
result =  [[x[0]] for x in resuld_df.values]# + [[x[0]] for x in resuld_df.values] + [[x[0]] for x in resuld_df.values]
result_data = np.array(result)
for x in range(0, len(result_data)):
	result_data[x] = result_data[x]
	result_data[x] = result_data[x] / 100.
result_data

In [ ]:
# Определяется модель Net - это нейронная сеть, 
# состоящая из слоя RNN (рекуррентная нейронная сеть) и полносвязного слоя. 
# Скрытый размерность RNN равен 16, а входной размерность равна 1.
# Модель использует функцию активации по умолчанию.
class Net(nn.Module):
# В конструкторе класса мы определяем наши слои: nn.RNN и nn.Linear. 
# Эти слои будут использоваться при проходе данных через модель. 
# Здесь мы указываем, что на входе имеем 1 признак, 
# размер скрытого состояния RNN равен 16, 
# а архитектура сети предполагает использование batch_first=True, 
# то есть размерность первой оси является размером пакета.
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=8, batch_first=True)
        self.fc = nn.Linear(8, 1)

# Функция forward принимает входной тензор x, 
# пропускает его через слой RNN, 
# получает выходной тензор и пропускает его через полносвязный слой, 
# чтобы получить выходной тензор.
    def forward(self, x):
        output, _ = self.rnn(x)
        x = self.fc(output[:, -1, :])
        return x.squeeze()  # удаляем избыточные размерности

In [ ]:
model = Net()
# определяется устройство, на котором будет проходить обучение модели (GPU или CPU).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
model.to(device)

In [ ]:
device

In [ ]:
# Определение функции потерь и оптимизатора
# Определяется функция потерь (criterion) - это среднеквадратичная ошибка (MSE).
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
# Определяется оптимизатор (optimizer) - 
# это алгоритм Adam с коэффициентом обучения 0.001, 
# который будет использоваться для обновления параметров модели.
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
import numpy as np
num_samples = 1000
data = []
for i in range(len(data_x)):
    tmp = torch.from_numpy(data_y[i].numpy()).reshape(8547,1)
    data.append(tmp.float().to(device),)

In [ ]:
# Генерируется синтетический набор данных для обучения - это список data, 
# который состоит из 1000 тензоров, 
# каждый из которых имеет случайный размер от 3 до 10 и 
# содержит случайные числа от 0 до 100. 
# Каждый тензор data заполняется нулями до максимальной длины, 
# чтобы все тензоры имели одинаковую длину. 
# Кроме того, создается тензор меток labels, 
# который состоит из 1000 случайных чисел от 0 до 100.
data_tensor = pad_sequence(data, batch_first=True)  # заполняем более короткие тензоры нулями

In [ ]:
data_tensor.to(device)

In [ ]:
# перемещение labels на устройство обусловлено тем, 
# что он используется в вычислении функции потерь, 
# которая определена на устройстве.
labels = torch.from_numpy(result_data).float().to(device)

In [ ]:
# изменяем размерность целевого тензора
# Аргумент -1 в данном случае означает, 
# что размерность должна быть вычислена автоматически 
# на основе других размерностей тензора.
labels = labels.view(-1)

In [ ]:
test_dataset = data_tensor
test_target = labels

In [ ]:
import statistics